<a href="https://colab.research.google.com/github/JRgreatest/Automated-Teller-Machine/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#take_photo에 대한 정의 파트, 코랩에선 서버에 사진이나 영상을 저장함
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode

def take_photo(filename='photo1.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      //캡쳐 버튼의 생성부분
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);
      //cavas 생성성
      const canvas = document.createElement('canvas');
      //크기 맞추기기
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      //이미지 그리기기
      canvas.getContext('2d').drawImage(video, 0, 0);
      //비디오 끄기기
      stream.getVideoTracks()[0].stop();
      //div삭제제
      div.remove();
      //파일 주소 반환환
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  #코드 실행한 결과 반환 값을 파이썬에 전달달
  data = eval_js('takePhoto({})'.format(quality))
  #웹 브라우저에서 데이터를 저장할 때 base64로 저장장
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
    #위에 저장된 def파트의 이름을 적는 것것
    filename = take_photo()
    print('Saved to {}'.format(filename))
    
    #가져온 이미지 보여주기
    display(Image(filename))
    #에러시 출력 파트
except Exception as err:
  print(str(err))


In [ ]:
import cv2

capture = cv2.VideoCapture(0) #0번 카메라, 외부 카메라 있으면 1번 2번 이런식
if capture.isOpened() == False: raise Exception('카메라 연결 안됨')

fps = 29.97
delay = round(1000/fps)
size = (640,360)
fourcc = cv2.VideoWriter_fourcc(*'DX50')

print('width * height:',size)
print('VideoWriterfourcc: %s' % fourcc)
print('delay: %2d ms' % delay)
print('fps : %.2f' % fps)

capture.set(cv2.CAP_PROP_ZOOM, 1)
capture.set(cv2.CAP_PROP_FOCUS, 0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, size[0])
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, size[1])

writer = cv2.VideoWriter("C:/Users/KIM/PycharmProjects/Drone/video/test.avi", fourcc, fps, size)#경로 설정 및 저장 이름
if writer.isOpened() == False: raise Exception('동영상 오픈 안됨')

while True:
    ret, frame = capture.read()
    if not ret : break
    if cv2.waitKey(delay) >= 0: break

    writer.write(frame)
    cv2.imshow("test", frame)

writer.release()
capture.release()